In [31]:
import os 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import time
from src.preprocessing import *
from src.master import*
#PATH
Path_Aircraft = 'data/1. AC characteristics/ACchar.xlsx'
Path_correspondance_Aircraft_Airport = 'Correspondance.pkl'
path_glossary = 'data/Glossary.xlsx'
Path_AirportData = 'data/0. Airport data/Airport_Data.csv'
Path_WeatherData = 'data/2. Weather data/weather_data_prep.csv'

In [4]:
#df = master_preprocessing_X(Path_AirportData, Path_WeatherData, Path_Aircraft, Path_correspondance_Aircraft_Airport)
df = cleaning_airport_df(Path_AirportData)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3209: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [26]:
X = get_df_of_obs1(df)
y = get_target_values(X)

In [27]:
col_numerical = []
col_category = ['carrier', 'acType','runway', 'stand']
col_to_drop=['sto','aldt', 'eibt','cibt', 'aibt', 'chocks_on', 'flight']

In [28]:
# droping useless column
X = X.drop(col_to_drop, axis=1)

# Scaling numerical values 
if len(col_numerical)>0:
    scaler = MinMaxScaler()
    X[col_numerical] = scaler.fit_transform(X[col_numerical])
    
# dummies transformation
X = pd.get_dummies(X, columns=col_category,drop_first=True)


In [29]:
## train/test split 
test_size = 1000
X_train, X_test = X[0:len(X)-test_size], X[len(X)-test_size:len(X)]
y_train, y_test = y[0:len(y)-test_size], y[len(y)-test_size:len(y)]


In [32]:
## Model
time_start = time.clock()
rf = RandomForestClassifier(n_estimators= 400, min_samples_split= 3, min_samples_leaf= 2, max_features= 'auto',max_depth= 50,bootstrap=True ,random_state=10)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)

time_elapsed = (time.clock() - time_start)
print(time_elapsed)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


305.3578767


In [76]:
TEST1 = pred/60
TEST2 = [x/60 for x in y_test[0].tolist()]
np.sqrt(sk.metrics.mean_squared_error(TEST1,TEST2))

4.421764353739353

In [ ]:
def drop_scaled_dummies(X):
    col_numerical = []
    col_category = ['carrier', 'acType','runway', 'stand']
    col_to_drop=['sto','aldt', 'eibt','cibt', 'aibt', 'chocks_on', 'flight']
    
    # droping useless column
    X = X.drop(col_to_drop, axis=1)

    # Scaling numerical values 
    if len(col_numerical)>0:
        scaler = MinMaxScaler()
        X[col_numerical] = scaler.fit_transform(X[col_numerical])
    
    # dummies transformation
    X = pd.get_dummies(X, columns=col_category,drop_first=True)
    return X

def train_test_split(X,test_size=1000):
    X_train, X_test = X[0:len(X)-test_size], X[len(X)-test_size:len(X)]
    y_train, y_test = y[0:len(y)-test_size], y[len(y)-test_size:len(y)]
    return X_train,X_test,y_train,y_test

def RF_modeling(X_train,X_test,y_train):
    rf = RandomForestClassifier(n_estimators= 400, min_samples_split= 3, min_samples_leaf= 2, max_features= 'auto',max_depth= 50,bootstrap=True ,random_state=10)
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    return pred

def RMSE(y_test,pred):
    pred_in_min = pred/60
    y_test_in_min = [x/60 for x in y_test[0].tolist()]
    np.sqrt(sk.metrics.mean_squared_error(TEST1,TEST2))


In [ ]:
df.isna().sum()/len(df)

In [ ]:
missing_aircraft = list(df[df['acType'].isna()]['flight'].unique())

In [ ]:
df[df['flight'].isin(missing_aircraft)].groupby('flight')['acType'].value_counts()

In [ ]:
df_aircraft = filtering_AC_charac(Path_correspondance_Aircraft_Airport,Path_Aircraft)

In [ ]:
df_aircraft[df_aircraft['Parking Area (WS x Length), sf'].isna()]

In [ ]:
['NaN','Boeing','767-400','Jet','2','D','IV','5','145','no winglets','170','201']
['NaN','McDonnell Douglas','md-90','Jet','2','C','III','']

In [ ]:
df_aircraft.head(50)